In [10]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta, time
import os

# =================================================
# CONFIG
# =================================================
INPUT_CSV = "blipTimesheet 26th Jan - 6th Feb(in).csv" 
OUTPUT_XLSX = "blipTimesheet_27Jan_onwards_.xlsx"

MIN_WORK = timedelta(hours=7, minutes=30)
random.seed(42)

# =================================================
# LOAD DATA
# =================================================
if not os.path.exists(INPUT_CSV):
    print(f"❌ Error: Input file '{INPUT_CSV}' not found.")
    exit()

df = pd.read_csv(INPUT_CSV, skiprows=1)

# Normalize headers
df.columns = (
    df.columns.str.strip()
              .str.lower()
              .str.replace(" ", "_")
              .str.replace(r"[()]", "", regex=True)
)

# =================================================
# BUILD DATETIMES (robust)
# =================================================
df["clock_in_dt"] = pd.to_datetime(
    df["clock_in_date"].astype(str) + " " + df["clock_in_time"].astype(str),
    errors="coerce"
)

df["clock_out_dt"] = pd.to_datetime(
    df["clock_out_date"].astype(str) + " " + df["clock_out_time"].astype(str),
    errors="coerce"
)

# Filter from 27 Jan onwards + weekdays
start_date = pd.to_datetime("2025-01-27")
df = df[df["clock_in_dt"].notna() & (df["clock_in_dt"] >= start_date)]
df = df[df["clock_in_dt"].dt.weekday < 5]

# =================================================
# HELPERS
# =================================================
def rand_time(h1, m1, h2, m2):
    s = random.randint(h1 * 3600 + m1 * 60, h2 * 3600 + m2 * 60)
    return time(s // 3600, (s % 3600) // 60, s % 60)

def rand_lunch_start():
    return rand_time(12, 0, 14, 30)

def get_team_from_group(g):
    team_cols = [c for c in g.columns if c.startswith("team")]
    if not team_cols:
        return np.nan
    
    shift = g[g["blip_type"].str.lower() == "shift"]
    for col in team_cols:
        if not shift.empty and shift[col].notna().any():
            return shift[col].dropna().iloc[0]
    for col in team_cols:
        if g[col].notna().any():
            return g[col].dropna().iloc[0]
    return np.nan

# =================================================
# PROCESS
# =================================================
rows = []

for (fn, ln, day), g in df.groupby(
    ["first_name", "last_name", df["clock_in_dt"].dt.date]
):
    base = g.iloc[0]
    team_val = get_team_from_group(g)
    notes = np.nan

    shifts = g[g["blip_type"].str.lower() == "shift"]
    breaks = g[g["blip_type"].str.lower() == "break"]

    if shifts.empty or pd.isna(shifts.iloc[0]["clock_in_dt"]):
        clock_in = rand_time(8, 55, 9, 10)
        clock_out = rand_time(17, 25, 17, 45)
        break_start = rand_lunch_start()
        break_td = timedelta(minutes=random.randint(30, 45), seconds=random.randint(0, 59))
        notes = "ADJUSTED (no valid shift)"
    else:
        s = shifts.iloc[0]
        clock_in = s["clock_in_dt"].time()
        clock_out = (
            s["clock_out_dt"].time()
            if pd.notna(s["clock_out_dt"])
            else rand_time(17, 25, 17, 45)
        )

        if breaks.empty:
            break_start = rand_lunch_start()
            break_td = timedelta(minutes=random.randint(30, 45), seconds=random.randint(0, 59))
            notes = "ADJUSTED (break added)"
        else:
            b = breaks.iloc[0]
            if pd.notna(b["clock_in_dt"]) and pd.notna(b["clock_out_dt"]):
                break_td = b["clock_out_dt"] - b["clock_in_dt"]
            else:
                break_td = timedelta(minutes=random.randint(30, 45))
            
            if timedelta(minutes=25) <= break_td <= timedelta(minutes=60):
                break_start = b["clock_in_dt"].time()
            else:
                break_start = rand_lunch_start()

        work_td = (
            datetime.combine(day, clock_out)
            - datetime.combine(day, clock_in)
            - break_td
        )

        if work_td < MIN_WORK:
            clock_out = rand_time(17, 25, 17, 45)
            notes = "ADJUSTED (working hours aligned)"

    # --- BUILD SHIFT ROW ---
    rows.append({
        "First Name": fn,
        "Last Name": ln,
        "Job Title": base["job_title"],
        "Team(s)": team_val,
        "Blip Type": "Shift",
        "Clock In Date": day,
        "Clock In Time": str(clock_in),
        "Clock In Location": base["clock_in_location"],
        "Clock Out Date": day,
        "Clock Out Time": str(clock_out),
        "Clock Out Location": base["clock_in_location"],
        "Total Duration": (datetime.combine(day, clock_out) - datetime.combine(day, clock_in)),
        "Total Excluding Breaks": (datetime.combine(day, clock_out) - datetime.combine(day, clock_in) - break_td),
        "Notes": notes,
    })

    # --- BUILD BREAK ROW ---
    rows.append({
        "First Name": fn,
        "Last Name": ln,
        "Job Title": base["job_title"],
        "Team(s)": team_val,
        "Blip Type": "Break",
        "Clock In Date": day,
        "Clock In Time": str(break_start),
        "Clock In Location": base["clock_in_location"],
        "Clock Out Date": day,
        "Clock Out Time": str((datetime.combine(day, break_start) + break_td).time()),
        "Clock Out Location": base["clock_in_location"],
        "Total Duration": break_td,
        "Total Excluding Breaks": timedelta(0),
        "Notes": notes,
    })

# =================================================
# EXPORT (Compatible with Dashboard)
# =================================================
final_df = pd.DataFrame(rows)

# Ensure data types are dashboard-safe
final_df["Clock In Date"] = pd.to_datetime(final_df["Clock In Date"]).dt.date
final_df["Clock Out Date"] = pd.to_datetime(final_df["Clock Out Date"]).dt.date
final_df["Total Duration"] = final_df["Total Duration"].astype(str)
final_df["Total Excluding Breaks"] = final_df["Total Excluding Breaks"].astype(str)

try:
    # 1. Try writing with openpyxl (Standard pandas dependency)
    # We use startrow=1 to leave room for the dummy header
    with pd.ExcelWriter(OUTPUT_XLSX, engine='openpyxl') as writer:
        final_df.to_excel(writer, sheet_name='Sheet1', startrow=1, index=False)
        # Add the dummy header manually
        writer.sheets['Sheet1']['A1'] = "Export generated for Dashboard (skiprows=1)"
    
    print("✔ FINAL dashboard-safe dataset created")
    print(f"✔ Output saved to {OUTPUT_XLSX}")

except ImportError:
    # 2. Fallback to CSV if no Excel writer is found
    print("⚠ 'openpyxl' module not found. Falling back to CSV.")
    csv_output = OUTPUT_XLSX.replace(".xlsx", ".csv")
    
    # Write the dummy header line first
    with open(csv_output, "w") as f:
        f.write("Export generated for Dashboard (skiprows=1)\n")
    
    # Append the dataframe
    final_df.to_csv(csv_output, mode="a", index=False)
    print(f"✔ Output saved to {csv_output} (Excel libraries were missing)")

except Exception as e:
    print(f"❌ unexpected error: {e}")

C:\Users\HarshMalhotra\AppData\Local\Temp\ipykernel_21216\1018999028.py:41: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["clock_out_dt"] = pd.to_datetime(


✔ FINAL dashboard-safe dataset created
✔ Output saved to blipTimesheet_27Jan_onwards_.xlsx
